# Config stuff

In [110]:

import ConnectionConfig as cc
from delta import DeltaTable
from datetime import datetime

In [111]:
cc.setupEnvironment()
spark = cc.startLocalCluster("dimSalesIncrementalLoad")
spark.getActiveSession()

## Incremental load

After the sales Rep dimension is filled for the first time, the logic to update the dimension has to be handled differently. A change of a record in the source system has to be handled as a change in the dimension. The SCD2 logic is used to handle this.

The SCD2 implementation requires a more complex transformation to correctly handle changes in the source files. For detailed information consult the comments in the code.
### Setting the parameters
The timestamp of the job is used to set the scd_end date of the previous record and the scd_start date of the new record.

In [112]:
run_timestamp =datetime.now() #The job runtime is stored in a variable

### Read existing dimension

In [114]:
dt_dimSalesRep = DeltaTable.forPath(spark,".\spark-warehouse\dimsalesrep")

dt_dimSalesRep.toDF().createOrReplaceTempView("dimSalesRep_current")

#DEBUG CODE TO SHOW CONTENT OF DIMENSION
spark.sql("select salesRepID, name, office, salesRepSK, md5  from dimSalesRep_current ").show()

+----------+--------------+-------------+--------------------+--------------------+
|salesRepID|          name|       office|          salesRepSK|                 md5|
+----------+--------------+-------------+--------------------+--------------------+
|         5|       J.Mosby|       Berlin|392eef95-7448-481...|fa85515824433037a...|
|         6|    B. Simpson|     New York|40945224-e95e-42f...|0ecb6ba5656492a5f...|
|         7|    B. Stinson|San Fransisco|2fa41c1a-4fb7-457...|e726b2d8dc0cf9a6f...|
|         8| L. Hofstadter|     Brussels|7b4e26d0-0ef4-479...|a2bbe52f8274b0f08...|
|         9|     S. Cooper|     Brussels|10c3e797-2cfb-490...|d85c73c9d03df0002...|
|         1|       Z. Jane|       London|20c5b991-3e36-4a3...|d5fbd19b14904dc4d...|
|         1|       Z. Jane|       Berlin|e1403167-d6bc-4e1...|00b409cd29c569266...|
|         2|P. Chapmanneke|       Berlin|4b7542d1-f37c-497...|ff8eb928f1004c797...|
|         3|      T. Crane|     New York|4aebfcb7-48c1-467...|6c062f95defda9

### Read source table


##### 1 READ SOURCE TABLE
Creating dataframe with source table (from operational system). Transformed to the dimension format.
The surrogate key is a uuid to be sure it's unique.
md5 hash is used to identify changes in the source table.
A view is created of the resulting dataframe to make it available for the next step.

In [115]:
cc.set_connection("tutorial_op")

#a. Reading from a JDBC source
df_operational_sales_rep = spark.read \
    .format("jdbc") \
    .option("driver" , "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "dbo.salesrep") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "salesRepID") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 20) \
    .load()

df_operational_sales_rep.createOrReplaceTempView("operational_sales_rep")

#b. Transforming the source to the dimension format
df_dim_sales_rep_new = spark.sql( "select uuid() as source_salesRepSK, \
                                        salesRepId as source_salesRepId, \
                                        name as source_name, \
                                        office as source_office, \
                                        md5(concat( name, office)) as source_md5 \
                                    from operational_sales_rep")

df_dim_sales_rep_new.createOrReplaceTempView("dimSalesRep_new")

#DEBUG CODE TO SHOW CONTENT OF SOURCE
#df_dim_sales_rep_new.printSchema()
#df_dim_sales_rep_new.show()
spark.sql("select * from dimSalesRep_new").show()
#df_dim_sales_rep.write.format("delta").mode("overwrite").saveAsTable("dimSalesRep")


+--------------------+-----------------+--------------+-------------+--------------------+
|   source_salesRepSK|source_salesRepId|   source_name|source_office|          source_md5|
+--------------------+-----------------+--------------+-------------+--------------------+
|d89dbdf9-eb37-445...|                1|       Z. Jane|       London|d5fbd19b14904dc4d...|
|a678917d-23db-4a1...|                2|P. Chapmanneke|       Berlin|ff8eb928f1004c797...|
|1c3a9418-111d-4c1...|                3|      T. Crane|     New York|6c062f95defda9dc3...|
|775d4941-765c-43e...|                4|     R. Geller|        Paris|6f34bd2f09f010ec1...|
|42260cc5-f0a0-45d...|                5|       J.Mosby|       Berlin|fa85515824433037a...|
|307a0130-974f-40a...|                6|    B. Simpson|     New York|0ecb6ba5656492a5f...|
|86051915-f668-465...|                7|    B. Stinson|San Fransisco|e726b2d8dc0cf9a6f...|
|afd18337-77c2-4bf...|                8| L. Hofstadter|     Brussels|a2bbe52f8274b0f08...|


##### 2 DETECT CHANGES
Dataframe to identify SCD2 changed rows.
First a join between SOURCE (operational system) and DIMENSION (dwh) is performed
   The md5 hash is used to identify differences.
   The list contains:
       - updated source-rows (the join finds a rowand the md5 is different)  and
       - new source-rows (the leftouter join does not find a row in the dimension (dwh.salesRepId is null)

In [116]:

detectedChanges=spark.sql(f"select * \
                          from dimSalesRep_new source \
                          left outer join dimSalesRep_current dwh on dwh.salesRepID == source.source_salesRepId and dwh.current == true \
                          where dwh.salesRepId is null or dwh.md5 <> source.source_md5")

detectedChanges.createOrReplaceTempView("detectedChanges")

#DEBUG CODE TO SHOW CONTENT OF DETECTED CHANGES
detectedChanges.show()


+-----------------+-----------------+-----------+-------------+----------+----------+----------+----+------+---------+-------+---+-------+
|source_salesRepSK|source_salesRepId|source_name|source_office|source_md5|salesRepSK|salesRepID|name|office|scd_start|scd_end|md5|current|
+-----------------+-----------------+-----------+-------------+----------+----------+----------+----+------+---------+-------+---+-------+
+-----------------+-----------------+-----------+-------------+----------+----------+----------+----+------+---------+-------+---+-------+




##### 3 TRANSOFRM TO UPSERTS
Before union: Every updated and new source-row requires the insertion of a new record in the SCD2 dimension. This new records starts at the runtime of the job and ends at the end of time (2100-12-12). Current is set to true.
Updated source-rows also require an update of the existing scd-fields. The scd_end date of the existing record is set to the runtime of the job. Current is set to false

In the next step, rows without mergeKey will be inserted in the dimension table and rows with mergekey will be updated in the dimension

In [117]:

df_upserts = spark.sql(f"select source_salesRepSK as salesRepSK,\
                                source_salesRepId as salesRepID,\
                                source_name as name,\
                                source_office as office,\
                                to_timestamp('{run_timestamp}') as scd_start, \
                                to_timestamp('2100-12-12','yyyy-MM-dd') as scd_end,\
                                source_md5 as md5,\
                                true as current\
                        from  detectedChanges\
                        union \
                        select  salesRepSK,\
                                salesRepId,\
                                name,\
                                office,\
                                scd_start,\
                                to_timestamp('{run_timestamp}') as scd_end,\
                                md5, \
                                false \
                                from detectedChanges \
                        where current is not null")

df_upserts.createOrReplaceTempView("upserts")

In [118]:

#DEBUG CODE TO SHOW CONTENT OF UPSERTS
spark.sql("select * from upserts").show()

+----------+----------+----+------+---------+-------+---+-------+
|salesRepSK|salesRepID|name|office|scd_start|scd_end|md5|current|
+----------+----------+----+------+---------+-------+---+-------+
+----------+----------+----+------+---------+-------+---+-------+

+--------------------+----------+--------------+-------------+--------------------+--------------------+--------------------+-------+
|          salesRepSK|salesRepID|          name|       office|           scd_start|             scd_end|                 md5|current|
+--------------------+----------+--------------+-------------+--------------------+--------------------+--------------------+-------+
|392eef95-7448-481...|         5|       J.Mosby|       Berlin| 1999-01-01 00:00:00| 2100-12-12 00:00:00|fa85515824433037a...|   true|
|40945224-e95e-42f...|         6|    B. Simpson|     New York| 1999-01-01 00:00:00| 2100-12-12 00:00:00|0ecb6ba5656492a5f...|   true|
|2fa41c1a-4fb7-457...|         7|    B. Stinson|San Fransisco| 19


#### PERFORM MERGE DIMSALESREP AND UPSERTS
merge looks for a matching dwh.salesRepID (in the dimension) for mergeKey
   - when a match is found (the dimension table contains a row where its salesRepId corresponds with one of the mergekeys)  -> perform update of row to close the period and set current to "false"
   - when no match is found (there is no salesRepID in the dimension because the mergeKey is null) -> perform an insert with the data from the updserts table (from the source). The scd-start is filled with the run_timestamp)

In [119]:
spark.sql("MERGE INTO dimSalesRep_current AS target \
          using upserts AS source ON target.salesRepID = source.salesRepID and source.current = false and target.current=true \
          WHEN MATCHED THEN UPDATE SET scd_end = source.scd_end, current = source.current  \
          WHEN NOT MATCHED THEN INSERT (salesRepSK, salesRepId, name, office, scd_start, scd_end, md5, current) values (source.salesRepSK, source.salesRepId, source.name, source.office, source.scd_start, source.scd_end, source.md5, source.current)")\

#DEBUG CODE TO SHOW CONTENT OF DIMENSION
dt_dimSalesRep.toDF().sort("salesRepID", "scd_start").show(100)


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

+--------------------+----------+--------------+-------------+--------------------+--------------------+--------------------+-------+
|          salesRepSK|salesRepID|          name|       office|           scd_start|             scd_end|                 md5|current|
+--------------------+----------+--------------+-------------+--------------------+--------------------+--------------------+-------+
|e1403167-d6bc-4e1...|         1|       Z. Jane|       Berlin| 1999-01-01 00:00:00|2023-09-20 11:37:...|00b409cd29c569266...|  false|
|20c5b991-3e36-4a3...|         1|       Z. Jane|       London|2023-09-20 11:37:...| 2100-12-12 00:00:00|d5fbd19b14904dc4d...|   true|
|4b7542d1-f37c-497...|         2|P. Chapmanneke|       Berlin| 1999-01-01 00:00:00| 2100-12-12 00:00:00|ff8eb928f1004c797...|   true|
|4aebfcb7-48c1-467...|         3|      T. Crane|     New York| 1999-01-01 00:00:00| 2100-12-12 00:00:00|6c062f95defda9dc3...|   true|
|1605101a-71cf-4bd...|         4|     R. Geller|        Paris|

## Delete the spark session

In [ ]:
#spark.stop()